In [1]:
################## preprocessing ###################
#################################################### 
####################################################
import pandas as pd
from os.path import isfile, join
import os
import logging
import argparse
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

date = '20220126'
basedir = '/home/lms/ms/poc'
rawdata_path = join(basedir, 'dataset')
procdata_path = join(basedir, 'deepfm', 'dataset', 'preprocess', 'rating')
procdata_path_r = join(basedir, 'deepfm', 'dataset', 'preprocess', 'rating')

In [2]:
user_data = pd.read_pickle(os.path.join(rawdata_path, 'user.pkl')) # unique id = 2047776
item_data = pd.read_pickle(os.path.join(rawdata_path, 'item.pkl'))
hist_data = pd.read_pickle(os.path.join(rawdata_path, 'history.pkl')) # unique id = 2038827

In [3]:
# 데이터 타입 통일
user_data = user_data.astype({'sa_id':'str'})
item_data = item_data.astype({'category_id':'str', 'category_name':'str', 'album_id':'str'})
hist_data = hist_data.astype({'sa_id':'str', 'album_id':'str', 'category_id':'str'})

In [4]:
item_columns = ['category_id','category_name','pr_info','price','release_date','run_time_ss']
item_data = item_data.dropna(subset=item_columns)

In [5]:
############################### 22.01.24 rating 신규 ##########################
# hist_data['rating_5'] = 5 # 5점 만들기
hist_data['rating_r'] = 2.5 + hist_data['watch_ratio'] * 2.5  # 나름대로 조작

In [6]:
hist_data.head(2)

,sa_id,album_id,rating,category_id,watch_ratio,current_rate,rating_r
0,205090929633,M01213B193PPV00,0.3731,B3054,0.6184,0.1278,4.046
1,205090929633,M01213H284PPV00,0.4639,B3054,0.8000,0.1278,4.500


In [7]:
# 차이 존재
hist_data = hist_data[['sa_id','album_id', 'rating_r']]

In [8]:
hist_data1 = hist_data.merge(item_data[['album_id','category_id']], on='album_id')

In [9]:
train_data = pd.DataFrame(hist_data1.groupby(by=['sa_id','category_id'], sort=False)['rating_r'].max()).reset_index()   # Maximun

In [15]:
train_data['rating_5'] = 5

In [10]:
item_columns = ['pr_info','price','release_date','run_time_ss']
category_data = item_data.groupby(by='category_id', sort=False)[item_columns].max()
category_data = pd.DataFrame(category_data).reset_index()

In [17]:
item_columns = ['category_id','pr_info','price','release_date','run_time_ss']
train_data = train_data.merge(user_data, on='sa_id')

In [18]:
train_data = train_data.merge(category_data[item_columns], on='category_id')

In [20]:
def encoding(dataset, columns):
    # label encoding
    for col_name in columns:
        codes, _ = dataset[col_name].factorize()            
        dataset[col_name + '_CODE'] = codes            

    return dataset

In [21]:
def minmax_scaling(dataset, columns):
    mms = MinMaxScaler(feature_range=(0, 1))
    dataset[columns] = mms.fit_transform(dataset[columns])

    return dataset

In [22]:
train_data = encoding(train_data, ['category_id','pr_info','price'])
train_data = encoding(train_data, ['sa_id'])

In [23]:
scale_columns = ['release_date', 'run_time_ss']
train_data['release_date'] = train_data['release_date'].replace('', '19900101')
train_data[scale_columns] = minmax_scaling(train_data[scale_columns], scale_columns)

/home/lms/anaconda3/envs/tf_deepctr/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/lms/anaconda3/envs/tf_deepctr/lib/python3.6/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [24]:
# for mapping data
mapping = train_data[['sa_id', 'category_id', 'category_id_CODE', 'sa_id_CODE']]
mapping.to_pickle(os.path.join(procdata_path_r, 'mapping.pkl'))

In [25]:
def save_code(dataset, user_column='sa_id', item_column='category_id'):
    user_map = dataset[[user_column, 
                        user_column+'_CODE', 
                        'seg_1', 
                        'seg_2', 
                        'seg_3', 
                        'seg_4']].drop_duplicates()
    item_map = dataset[[item_column, 
                        item_column+'_CODE', 
                        'price_CODE', 
                        'pr_info_CODE', 
                        'release_date', 
                        'run_time_ss']].drop_duplicates()

    user_map.to_pickle(os.path.join(procdata_path_r, 'user.pkl'))
    item_map.to_pickle(os.path.join(procdata_path_r, 'item.pkl'))
            

def save_history(dataset):
    dataset.to_pickle(os.path.join(procdata_path_r, 'hist.pkl'))

In [26]:
save_code(train_data)

In [27]:
save_columns = ['sa_id_CODE',
                'category_id_CODE',
                'rating_r',
                'rating_5',
                'seg_1',
                'seg_2',
                'seg_3',
                'seg_4',
                'pr_info_CODE',
                'price_CODE',
                'release_date',
                'run_time_ss'
               ]
save_history(train_data[save_columns])  